In [15]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib import font_manager as fm
plt.rcParams["font.family"] = 'Malgun Gothic'
plt.rcParams['figure.figsize'] = [10, 5]

from pmdarima import auto_arima

import numpy as np
import os
import openpyxl
import xlsxwriter
import sys

# from guppy import hpy
from tqdm import tqdm_notebook

In [16]:
ext_m = pd.read_csv("D:\\시세모니터링\\융합분석\\DATA\\merge_ext_20200612.csv")
ext_m

,date,지역명,aptprice_all_sum,CD(91일),CMA(수시형),gold,hang-sen-40,kosdaq,KOSDAQ_거래대금,kospi,...,u.s.-30-year-bond-yield,us-30,us-dollar-index,us-spx-500,us_bs_rate,volatility-s-p-500,"콜금리(1일, 전체거래)",통안증권(1년),통안증권(2년),"회사채(3년,AA-)"
0,2007-01-05,전국,1.367448e+11,4.87,4.44,604.9,20211.28,602.35,20625,1385.76,...,4.739,12398.01,84.400,1409.71,5.250,12.14,4.67,4.930,4.940,5.250
1,2007-01-05,강원도,1.070694e+09,4.87,4.44,604.9,20211.28,602.35,20625,1385.76,...,4.739,12398.01,84.400,1409.71,5.250,12.14,4.67,4.930,4.940,5.250
2,2007-01-05,경기도,4.684591e+10,4.87,4.44,604.9,20211.28,602.35,20625,1385.76,...,4.739,12398.01,84.400,1409.71,5.250,12.14,4.67,4.930,4.940,5.250
3,2007-01-05,경상남도,3.523327e+09,4.87,4.44,604.9,20211.28,602.35,20625,1385.76,...,4.739,12398.01,84.400,1409.71,5.250,12.14,4.67,4.930,4.940,5.250
4,2007-01-05,경상북도,1.710930e+09,4.87,4.44,604.9,20211.28,602.35,20625,1385.76,...,4.739,12398.01,84.400,1409.71,5.250,12.14,4.67,4.930,4.940,5.250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40177,2020-06-05,파주시,2.043878e+09,0.79,0.24,1683.0,24770.41,749.31,86562,2181.87,...,1.669,27110.98,96.922,3193.93,0.125,24.52,0.47,0.674,0.799,2.224
40178,2020-06-05,평택시,2.125171e+09,0.79,0.24,1683.0,24770.41,749.31,86562,2181.87,...,1.669,27110.98,96.922,3193.93,0.125,24.52,0.47,0.674,0.799,2.224
40179,2020-06-05,포천시,1.834887e+08,0.79,0.24,1683.0,24770.41,749.31,86562,2181.87,...,1.669,27110.98,96.922,3193.93,0.125,24.52,0.47,0.674,0.799,2.224
40180,2020-06-05,하남시,2.822876e+09,0.79,0.24,1683.0,24770.41,749.31,86562,2181.87,...,1.669,27110.98,96.922,3193.93,0.125,24.52,0.47,0.674,0.799,2.224


In [17]:
ext_m.columns

Index(['date', '지역명', 'aptprice_all_sum', 'CD(91일)', 'CMA(수시형)', 'gold',
       'hang-sen-40', 'kosdaq', 'KOSDAQ_거래대금', 'kospi', 'KOSPI_거래대금시간외',
       'KOSPI_시가총액', 'kr_bs_rate', 'MMF(7일)', 'nq-100', 'shanghai-composite',
       'smallcap-2000', 'south-korea-1-year-bond-yield',
       'south-korea-2-year-bond-yield', 'south-korea-3-year-bond-yield',
       'south-korea-5-year-bond-yield', 'u.s.-10-year-bond-yield',
       'u.s.-30-year-bond-yield', 'us-30', 'us-dollar-index', 'us-spx-500',
       'us_bs_rate', 'volatility-s-p-500', '콜금리(1일, 전체거래)', '통안증권(1년)',
       '통안증권(2년)', '회사채(3년,AA-)'],
      dtype='object')

In [18]:
sgg_lst = pd.unique(ext_m["지역명"])
sgg_lst = sgg_lst[:25] # 도시

In [19]:
ext_m[ext_m["지역명"] == "전국"].iloc[:,2:].corr()["aptprice_all_sum"].sort_values(ascending=False)

aptprice_all_sum                 1.000000
nq-100                           0.975938
us-30                            0.932675
us-spx-500                       0.925720
smallcap-2000                    0.876460
KOSPI_시가총액                       0.818344
us-dollar-index                  0.786860
KOSDAQ_거래대금                      0.736979
kospi                            0.640145
hang-sen-40                      0.609901
gold                             0.472248
kosdaq                           0.472103
KOSPI_거래대금시간외                    0.180816
us_bs_rate                      -0.082349
shanghai-composite              -0.083375
volatility-s-p-500              -0.195408
u.s.-10-year-bond-yield         -0.589557
콜금리(1일, 전체거래)                   -0.731039
kr_bs_rate                      -0.738718
CD(91일)                         -0.759233
회사채(3년,AA-)                     -0.768363
u.s.-30-year-bond-yield         -0.773641
통안증권(1년)                        -0.786510
south-korea-1-year-bond-yield   -0

In [20]:
ext_m[ext_m["지역명"] == "강남구"].iloc[:,2:].corr()["aptprice_all_sum"].sort_values(ascending=False)

aptprice_all_sum                 1.000000
nq-100                           0.882894
us-30                            0.840422
us-spx-500                       0.823183
KOSDAQ_거래대금                      0.732148
smallcap-2000                    0.716249
us-dollar-index                  0.639912
KOSPI_시가총액                       0.595128
hang-sen-40                      0.582791
kosdaq                           0.495214
kospi                            0.438295
KOSPI_거래대금시간외                    0.312691
gold                             0.270165
us_bs_rate                       0.219668
shanghai-composite               0.079436
volatility-s-p-500              -0.020700
u.s.-10-year-bond-yield         -0.315021
회사채(3년,AA-)                     -0.473284
콜금리(1일, 전체거래)                   -0.485974
CD(91일)                         -0.492848
kr_bs_rate                      -0.494510
통안증권(1년)                        -0.509998
통안증권(2년)                        -0.521177
south-korea-1-year-bond-yield   -0

In [21]:
# ext_m_2 = pd.read_csv("D:\\시세모니터링\\융합분석\\외생변수_2_202049.csv")
# apt_ext_2 = pd.merge(aptaprice, ext_m_2, how = 'inner')
# apt_ext_2

In [ ]:
# 결과 저장
## 파일명 및 경로설정
import datetime
import timedelta
dt = datetime.datetime.now()
fl_dt = str(dt.year) + str(dt.month) + str(dt.day)
fl_nm = "result_all_data_" + fl_dt
sv_path = 'D:/시세모니터링/융합분석/산출문서/'

## 파일 쓰기 시작
workbook = xlsxwriter.Workbook(sv_path + fl_nm + '.xlsx')
worksheet = workbook.add_worksheet()

for list_idx, sgg_nm in enumerate(tqdm_notebook(sgg_lst)):
    train_dt = ext_m[ext_m["지역명"] == sgg_nm].sort_index(ascending=True)
    
    print(sgg_nm ,"Correlation :")
    print("\n")
    print(ext_m[(ext_m["지역명"] == sgg_nm)].corr()["aptprice_all_sum"].sort_values(ascending=False))
    print("\n")

    ## http://www.alkaline-ml.com/pmdarima/0.9.0/modules/generated/pyramid.arima.auto_arima.html

    stepwise_model = auto_arima(train_dt[["aptprice_all_sum"]], 
#                                 exogenous=train_dt[['south-korea-3-year-bond-yield', 'south-korea-5-year-bond-yield','u.s.-30-year-bond-yield']],
                                exogenous=train_dt[["us-30","KOSPI_시가총액","us-spx-500", "u.s.-30-year-bond-yield","MMF(7일)","CMA(수시형)"]],
                                start_p=0, start_q=0,
                               max_p=48, max_q=48,
                               m=48,
                               start_P=0, 
                               start_Q=0,
                               seasonal=True,
                               d=2, 
                               D=1,
                               trace=False,
    #                            maxiter = 100,
                               error_action='ignore',  
                               suppress_warnings=False, 
    #                            seasonal_test = ''
                               stepwise=True
                               )



    train = train_dt["aptprice_all_sum"]
    stepwise_model.fit(train)


    # 오늘날짜 기준
    dt_p = dt + datetime.timedelta(days=1)

    date_all = pd.date_range(str(dt_p.year) + "-" + str(dt_p.month) + "-" + str(dt_p.day + 1), "2020-12-31")
    date_all = date_all.to_frame().reset_index()
    # date_all.columns = ["date"]
    date_all = date_all.rename(columns={0 : "date"})
    date_all = date_all.drop(columns=["index"])
#     date_all = date_all[date_all["date"].dt.weekday_name == "Friday"]
    date_all = date_all[date_all["date"].dt.day_name() == "Friday"]

    future_forecast = stepwise_model.predict(n_periods=len(date_all) 
    #                                          exogenous = test[['usbond_3y', 'rate','kospi_price','소비자물가_intp', 'gold', 'kr_price_idx_intp','b_rate_us','kosdaq_price']]
                                            )

    # future_forecast = pd.DataFrame(future_forecast, columns=['ARIMA_PRD'])
    future_forecast = pd.DataFrame(future_forecast, index = date_all["date"].astype(str),columns=['aptprice_all_sum'])
    future_forecast["지역명"] = sgg_nm
    future_forecast = future_forecast.reset_index()
#     future_forecast = future_forecast.drop(columns=["index"])
    future_forecast = future_forecast[["date",'지역명','aptprice_all_sum']]
    future_forecast = future_forecast.rename(columns={"aptprice_all_sum" : 'aptprice_all_sum_prd'})
    
    ## 산출문서 값 변경 : 예측값 => 예측 변동율
    future_forecast_r = future_forecast
    train_dt_s = train_dt[["date","지역명","aptprice_all_sum"]][-1:]
    train_dt_s = train_dt_s.rename(columns={'aptprice_all_sum' : 'aptprice_all_sum_prd'})
    
    future_forecast_r = pd.concat([train_dt_s,future_forecast_r], axis = 0)
    future_forecast_r["change"] = (future_forecast_r["aptprice_all_sum_prd"].diff() / future_forecast_r["aptprice_all_sum_prd"]) * 100
    future_forecast_r = future_forecast_r[1:]
    
    prd_res = pd.concat([train_dt,future_forecast], axis = 0)
    prd_res = prd_res.set_index(pd.to_datetime(prd_res["date"], format = '%Y-%m-%d'))
    prd_res = prd_res.drop(columns=["date"])
    
    # 그래프 출력
#     pd.concat([prd_res["aptprice_all_sum"],prd_res["ARIMA_PRD"]],axis=1).plot()
    # 1. 시가총액 + kospi 그래프
    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()
    
    plt.plot(prd_res[:ext_m["date"].iloc[-1]].index, prd_res["aptprice_all_sum"][:ext_m["date"].iloc[-1]].values, label = sgg_nm)
    plt.plot(prd_res[ext_m["date"].iloc[-1]:].index, prd_res["aptprice_all_sum_prd"][ext_m["date"].iloc[-1]:].values, label = sgg_nm)
#     ax1.plot(ext_m["date"], ext_m["aptprice_all_sum"], label = sgg_nm)
#     ax2.plot(ext_m["date"], ext_m["u.s.-30-year-bond-yield"] , color = 'y')
    plt.title(sgg_nm + " 시가총액")
    
#     # 1. 시가총액 + gold 그래프
#     fig, ax3 = plt.subplots()
#     ax4 = ax3.twinx()
    
#     ax3.plot(prd_res["date"], prd_res["aptprice_all_sum"], label = sgg_nm,color = 'b')
#     ax4.plot(ext_m["date"],ext_m["gold"], label = ext_m.iloc[:,2:].corr()["aptprice_all_sum"].sort_values(ascending=False).index[2] , color = 'y')
#     plt.title(sgg_nm + " 시가총액 " + apt_ext.iloc[:,2:].corr()["aptprice_all_sum"].sort_values(ascending=False).index[2])

    # 결과 쓰기
    frow = 1
    crow = 1

    date_lst = date_all["date"].astype(str).tolist()

    ## 첫번째열로 시작할때만 날짜 추가
    ### 최초에만 실행
    if list_idx == 0:
        for lst1 in date_lst:
            worksheet.write(frow,0,lst1)
            frow += 1
    ### 최초 날짜열 끝
    ## 추가 종료

    # 시군구명 column(가로로 추가)
    worksheet.write(0,list_idx + 1,sgg_nm)

    content = future_forecast_r.change.values.tolist()
    
    # 예측값 구별로 넣기
    row = 1
    for j in content:
        worksheet.write(row, list_idx + 1, j)
        row += 1
        ## 결과 쓰기 종료
    

#     print("제거전 메모리 : ",hpy().heap())
    print(list_idx + 1,"회차 완료 : ", sgg_nm)
    print("\n")
    

#     print("stepwise_model : ",sys.getsizeof(stepwise_model))
#     print("train_dt : ", sys.getsizeof(train_dt))
#     print("train : ", sys.getsizeof(train))
#     print("future_forecast : ", sys.getsizeof(future_forecast))
    
    del train_dt
    del train
    del stepwise_model
    del future_forecast
    del future_forecast_r
#     print("제거 후 메모리 : ",hpy().heap())


workbook.close()

c:\users\jhpark1\anaconda3\envs\tc\lib\site-packages\ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


전국 Correlation :


aptprice_all_sum                 1.000000
nq-100                           0.975938
us-30                            0.932675
us-spx-500                       0.925720
smallcap-2000                    0.876460
KOSPI_시가총액                       0.818344
us-dollar-index                  0.786860
KOSDAQ_거래대금                      0.736979
kospi                            0.640145
hang-sen-40                      0.609901
gold                             0.472248
kosdaq                           0.472103
KOSPI_거래대금시간외                    0.180816
us_bs_rate                      -0.082349
shanghai-composite              -0.083375
volatility-s-p-500              -0.195408
u.s.-10-year-bond-yield         -0.589557
콜금리(1일, 전체거래)                   -0.731039
kr_bs_rate                      -0.738718
CD(91일)                         -0.759233
회사채(3년,AA-)                     -0.768363
u.s.-30-year-bond-yield         -0.773641
통안증권(1년)                        -0.786510
south-korea-1-y

c:\users\jhpark1\anaconda3\envs\tc\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
c:\users\jhpark1\anaconda3\envs\tc\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
c:\users\jhpark1\anaconda3\envs\tc\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
c:\users\jhpark1\anaconda3\envs\tc\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
c:\users\jhpark1\anaconda3\envs\tc\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimizat

1 회차 완료 :  전국


강원도 Correlation :


aptprice_all_sum                 1.000000
nq-100                           0.932993
us-30                            0.899788
us-spx-500                       0.899709
smallcap-2000                    0.898726
KOSPI_시가총액                       0.889296
us-dollar-index                  0.810326
kospi                            0.716940
KOSDAQ_거래대금                      0.657738
hang-sen-40                      0.572747
gold                             0.534286
kosdaq                           0.431922
KOSPI_거래대금시간외                    0.060294
shanghai-composite              -0.163046
us_bs_rate                      -0.261428
volatility-s-p-500              -0.301896
u.s.-10-year-bond-yield         -0.698917
콜금리(1일, 전체거래)                   -0.815789
kr_bs_rate                      -0.821578
u.s.-30-year-bond-yield         -0.841136
CD(91일)                         -0.855291
회사채(3년,AA-)                     -0.880496
통안증권(1년)                        -0.88555

c:\users\jhpark1\anaconda3\envs\tc\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
c:\users\jhpark1\anaconda3\envs\tc\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
c:\users\jhpark1\anaconda3\envs\tc\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:963: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\users\jhpark1\anaconda3\envs\tc\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:975: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\users\jhpark1\anaconda3\envs\tc\lib\site-packages\statsmod